# Chapter 4: Statistics

**Mathematics for Machine Learning (2025)**
Teaching Team

Lecturers: 
- Eleonora Giunchiglia (Department of Electrical and Electronic Engineering)
- Felipe Tobar (Department of Mathematics)

Graduate Teaching Assistants (GTAs)
- Camilo Carvajal Reyes (Department of Mathematics)
- Joshua Ong (Department of Electrical and Electronic Engineering)

In [ ]:
# plot l(\theta) and MLE

In [ ]:
# prior, likelihood, posterior

In [ ]:
# Posterior concentration with Dirichlet prior

In [ ]:
# Example of posterior prediction vs plug-in predition

Let $X$ be a random variable in $\mathbb{R}^p$ and $Y$ a real-valued random variable. Consider the linear regression model:

$$ Y = \beta_0 + \beta^T X + \epsilon,\quad \epsilon \sim \mathcal{N}(0, \sigma^2),$$

with $\beta = (\beta_1, \dots, \beta_p)^T$. We assume $\sigma^2$ is known. We are interested in the Bayesian estimation of the parameters $\theta = (\beta_0, \beta_1, \dots, \beta_p)$. We assume that the distribution of $X$ does not depend on $\theta$. Let $(x_i, y_i)_{1 \le i \le N}$ be data.

> Exercise: simulate the data considering $X\sim\mathcal{U}[0,1]$ and for $\beta_0=0$ and $\beta=1$. Plot the resulting r.v. $Y$ for different noise levels $\sigma^2$.

> Exercise: Show that the **maximum a posteriori** (MAP) estimator with a constant uniform prior (which does not depend on $\theta$) is equivalent to the maximum likelihood estimator.

...

Suppose we use the following prior:

$$ p(\beta) = \frac{1}{\sqrt{2\pi},\tau} , e^{-\frac{\beta^2}{2\tau^2}}.$$

> Exercise: Compute the maximum a posteriori estimator. Identify which known estimator it corresponds to. Implement a method that returns the MAP given the simulated data.

Consider the cases:

- $\tau \to \infty$
- $\tau \to 0$

> Exercise: Discuss what happens in each case, both in terms of the estimator found and in Bayesian terms (that is, think about the shape of the priors in both cases). Choose certain values of $\tau$ in both regimes, as well as a reference value $\tau=1$. Plot the priors and posterior in each case and comment.